Installed Langchain:

pip install -qU "langchain[openai]"

pip install -U langchain-community

pip install pypdf

pip install sentence-transformers

pip install hf_xet

In [1]:
import tqdm as notebook_tqdm

In [2]:
#Load the LargeLanguageModel

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import os

from transformers import AutoConfig

tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2")


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

config = AutoConfig.from_pretrained("microsoft/phi-2")

# 4. Load model with offloading enabled
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-2",
    device_map="auto",  # OR pass custom map below if needed
    quantization_config=bnb_config,
    offload_folder="./offload",  # Where to store offloaded layers
)


c:\Users\varma\anaconda3\envs\gpu-phi2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.83s/it]


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import os

from transformers import AutoConfig

tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2")


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

config = AutoConfig.from_pretrained("microsoft/phi-2")

# 4. Load model with offloading enabled
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-2",
    device_map="auto",  # OR pass custom map below if needed
    quantization_config=bnb_config,
    offload_folder="./offload",  # Where to store offloaded layers
)


Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.00s/it]


In [4]:
from langchain.document_loaders import PyPDFLoader

# Load the PDF file
pdf_loader = PyPDFLoader("trainingdata.pdf")
documents = pdf_loader.load()

# Display the loaded documents
print(documents)

[Document(metadata={'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'creator': 'PyPDF', 'creationdate': 'D:20250505060459', 'source': 'trainingdata.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content="KarPer: Detailed Knowledge Base of Karteek Varma\n1. Biographical Background\nKarteek Varma Pericharla is a dedicated and ambitious Artificial Intelligence student currently pursuing his\nMaster of Science in Computer Science with a specialization in Artificial Intelligence at Illinois Institute of\nTechnology (IIT), Chicago. He is expected to graduate in May 2025. This marks his second master?s degree,\nbuilding upon his prior MBA in Business Analytics, Artificial Intelligence, and Machine Learning from Woxsen\nUniversity, Hyderabad. His fascination with AI was sparked during his techno-managerial journey at Woxsen,\nprompting him to deepen his technical expertise through a second graduate program in the United States.\nBefore his postgraduate studies, he earned

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# Split the documents into chunks
chunks = text_splitter.split_documents(documents)

# Display the chunks
print(chunks)

[Document(metadata={'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'creator': 'PyPDF', 'creationdate': 'D:20250505060459', 'source': 'trainingdata.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content="KarPer: Detailed Knowledge Base of Karteek Varma\n1. Biographical Background\nKarteek Varma Pericharla is a dedicated and ambitious Artificial Intelligence student currently pursuing his\nMaster of Science in Computer Science with a specialization in Artificial Intelligence at Illinois Institute of\nTechnology (IIT), Chicago. He is expected to graduate in May 2025. This marks his second master?s degree,\nbuilding upon his prior MBA in Business Analytics, Artificial Intelligence, and Machine Learning from Woxsen\nUniversity, Hyderabad. His fascination with AI was sparked during his techno-managerial journey at Woxsen,\nprompting him to deepen his technical expertise through a second graduate program in the United States.\nBefore his postgraduate studies, he earned

In [6]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

db = Chroma.from_documents(chunks, embedding=embedding_model, persist_directory="C:\\Users\\varma\\my-site\\src\\chroma_store", collection_name="karteek_context")

db.persist()

C:\Users\varma\AppData\Local\Temp\ipykernel_21904\3006842507.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
C:\Users\varma\AppData\Local\Temp\ipykernel_21904\3006842507.py:8: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


The cells below is only for test inference, and do not need once deployed unless to test

In [7]:
query = "Tell me about his projects"
retrieved_docs = db.similarity_search(query, k=2)

#Get the context as plain text
context = "\n\n".join([doc.page_content for doc in retrieved_docs])

print("Context: ", context)


Context:  KarPer: Detailed Knowledge Base of Karteek Varma
- NLP: Sentiment analysis, Text summarization, GPT fine-tuning, BLEU/ROUGE/Perplexity metrics,
Word2Vec, TextBlob, NLTK
- Data Visualization: Tableau, Power BI, Matplotlib, Seaborn, ggplot
- Data Engineering: AWS pipelines, graph-based processing, feature engineering
- Computer Vision: Object detection, facial recognition, real-time video analysis
- Tools: Git, CI/CD pipelines, Power Vision (for HMI)
5. Key Projects & Achievements
1. **Flood Prediction using Transformer Models**: Karteek led the development of an AI-based flood risk
predictor using satellite imagery, transformer ML models, and geospatial data to help urban planners prepare
mitigation strategies. This project earned 3rd place in the Grainger Computing Innovation Prize at Illinois
Tech.
2. **AI Stock Trading Agent**: Designed a Double Deep Q-Learning agent that used LSTM networks to

KarPer: Detailed Knowledge Base of Karteek Varma
- NLP: Sentiment analysis, Text

In [8]:
collections = db._client.list_collections()
print([c.name for c in collections])


['karteek_context']


In [9]:
# prompt = f"""Context:
# {context}
#
# Answer the following question strictly using the context.
#
# Question: {query}
# Answer:"""

In [10]:
# input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)

# with torch.no_grad():
#     output_ids = model.generate(
#         input_ids,
#         max_new_tokens=150,
#         pad_token_id=tokenizer.eos_token_id,
#         eos_token_id=tokenizer.eos_token_id,
#     )

# Slice to get only the generated part (excluding prompt)
# generated_tokens = output_ids[0][input_ids.shape[-1]:]

# Decode only generated tokens
# response = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()
# response = response.split("Exercise")[0].strip()
# response = response.split("Question")[0].strip()

# print(response)
